# LINEAR PIZZA OVEN CHALLENGE

---

### Sources:
- S1: Camera that recognizes the topping [ws://streams:9400](ws://streams:9400)
- S2: Oven thermometer [ws://streams:9500](ws://streams:9500)

---

In [1]:
from rsplib import RSPEngine, RSPPublisher, Stream, rdf_table,load_graph, accessURL

## Publish the streams generated by the two sensors

In [193]:
###Create stream from both sensors
streamhub = RSPPublisher("http://streamhub:9292/streamhub")

In [194]:
##Publication query
vocals1 = '''
BASE  <http://streamhub:9292/streamhub>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX frmt: <http://www.w3.org/ns/formats/>
PREFIX vocals: <http://w3id.org/rsp/vocals#>
PREFIX vsd: <http://w3id.org/rsp/vocals-sd#>
REGISTER STREAM :pizzastream
FROM SOURCE <ws://streams:9400>
WHERE {

           {this} a vocals:StreamDescriptor .

           {publisher} a vsd:PublishingService ;
                         vsd:hasFeature vsd:replaying ;
                         vsd:resultFormat frmt:JSON-LD .

           :PizzaEndpoint a vocals:StreamEndpoint ;
                            dcat:title "A Pizza Stream Endpoint"^^xsd:string ;
                            dcat:description "Streaming endpoint to consume Colors via WebSocket"^^xsd:string ;
                            dcat:license <https://creativecommons.org/licenses/by-nc/4.0/> ;
                            dcat:format frmt:JSON-LD ;
                            dcat:accessURL {source} ;
                            vsd:publishedBy {publisher} .

           {stream} a vocals:RDFStream ;
                    dcat:title "Color Stream"^^xsd:string ;
                    dcat:description "Stream of colors"^^xsd:string ;
                    dcat:publisher {publisher} ;
                    dcat:landingPage <https://example.org/rw/colors/> ;
                    vocals:hasEndpoint :PizzaEndpoint  .

}'''

In [195]:
streamhub.publish(vocals1)

'{"error":"Duplicate Resource", "data": { "key": "http://streamhub:9292/streamhub/streams/pizzastream", "class": "String"}}'

In [7]:
###Publication query 2
vocals2 = '''
BASE  <http://streamhub:9292/streamhub>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX frmt: <http://www.w3.org/ns/formats/>
PREFIX vocals: <http://w3id.org/rsp/vocals#>
PREFIX vsd: <http://w3id.org/rsp/vocals-sd#>
REGISTER STREAM :temperaturestream
FROM SOURCE <ws://streams:9500>
WHERE {

           {this} a vocals:StreamDescriptor .

           {publisher} a vsd:PublishingService ;
                         vsd:hasFeature vsd:replaying ;
                         vsd:resultFormat frmt:JSON-LD .

           :TemperatureEndpoint a vocals:StreamEndpoint ;
                            dcat:title "A Temperature Stream Endpoint"^^xsd:string ;
                            dcat:description "Streaming endpoint to consume Colors via WebSocket"^^xsd:string ;
                            dcat:license <https://creativecommons.org/licenses/by-nc/4.0/> ;
                            dcat:format frmt:JSON-LD ;
                            dcat:accessURL {source} ;
                            vsd:publishedBy {publisher} .

           {stream} a vocals:RDFStream ;
                    dcat:title "Color Stream"^^xsd:string ;
                    dcat:description "Stream of colors"^^xsd:string ;
                    dcat:publisher {publisher} ;
                    dcat:landingPage <https://example.org/rw/colors/> ;
                    vocals:hasEndpoint :TemperatureEndpoint  .

}'''

In [8]:
streamhub.publish(vocals2)

{
    "@id": "streams/temperaturestream",
    "@type": "vocals:StreamDescriptor",
    "dcat:dataset": {
        "@id": "streams/temperaturestream",
        "@type": "vocals:RDFStream",
        "dcat:description": "Stream of colors",
        "dcat:landingPage": "https://example.org/rw/colors/",
        "dcat:publisher": "http://streamhub:9292/streamhub",
        "dcat:title": "Color Stream",
        "vocals:hasEndpoint": [
            [
                {
                    "@id": "http://streamhub:9292/streamhub/TemperatureEndpoint",
                    "@type": "http://w3id.org/rsp/vocals#StreamEndpoint",
                    "dcat:title": "A Temperature Stream Endpoint",
                    "dcat:description": "Streaming endpoint to consume Colors via WebSocket",
                    "dcat:license": "https://creativecommons.org/licenses/by-nc/4.0/",
                    "dcat:format": "http://www.w3.org/ns/formats/JSON-LD",
                    "dcat:accessURL": "ws://streams:9500",
    

In [9]:
streamhub.lists()

[http://streamhub:9292/streamhub/streams/temperaturestream,
 http://streamhub:9292/streamhub/streams/pizzastream]

#### Use this to delete the streams before closing...

In [8]:
streamhub.delete('pizzastream')
streamhub.delete('temperaturestream')
streamhub.lists()

[]


---

## IT'S QUERY TIME

### Initialize the RSPEngine

In [205]:
jasper = RSPEngine("http://jasper:8181/jasper")

In [11]:
jasper.streams()

[]

In [16]:
jasper.tasks()

[]

## Define the query

### Get the knowledge graph with Ontop from Carl's database
- Connect to the ontop container via SSH
- Run   
`./ontop materialize --format rdfxml -m input/cooking-mapping.ttl -p input/cooking.properties -o input/cooking-knowledge-graph.rdf`  
to export the graph built from Carl's relational database to an rdf file (Available at [http://localhost:8080/files/cooking-knowledge-graph.rdf](http://localhost:8080/files/cooking-knowledge-graph.rdf))

#### Generated RDF file `cooking-knowledge-graph.rdf` below

In [8]:
graph_file = open('./cooking-knowledge-graph.rdf', 'r')
text = graph_file.read()
print(text)

<?xml version="1.0" encoding="UTF-8"?>
<rdf:RDF
	xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">

<rdf:Description rdf:about="http://www.co-ode.org/ontologies/pizza/pizza.owl#Margherita">
	<tempAvg xmlns="http://linkeddata.stream/ontologies/cooking#" rdf:datatype="http://www.w3.org/2001/XMLSchema#decimal">280</tempAvg>
</rdf:Description>

<rdf:Description rdf:about="http://www.co-ode.org/ontologies/pizza/pizza.owl#American">
	<tempAvg xmlns="http://linkeddata.stream/ontologies/cooking#" rdf:datatype="http://www.w3.org/2001/XMLSchema#decimal">270</tempAvg>
</rdf:Description>

<rdf:Description rdf:about="http://www.co-ode.org/ontologies/pizza/pizza.owl#Margherita">
	<tempStd xmlns="http://linkeddata.stream/ontologies/cooking#" rdf:datatype="http://www.w3.org/2001/XMLSchema#decimal">10</tempStd>
</rdf:Description>

<rdf:Description rdf:about="http://www.co-ode.org/ontologies/pizza/pizza.owl#American">
	<tempStd xmlns="http://linkeddata.stream/ontologies/cooking#" rdf:datatype="ht

### Check the Pizza Ontology
Check the [Pizza Ontology](https://protege.stanford.edu/ontologies/pizza/pizza.owl) to write the query

PREFIX is not supported, I have to use it statically...

#### List of useful PREFIXs I'll need:
- PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
- PREFIX pizza: <http://www.co-ode.org/ontologies/pizza/pizza.owl#>
- PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
- PREFIX owl: <http://www.w3.org/2002/07/owl#>
- PREFIX sosa: http://www.w3.org/ns/sosa/
- http://localhost:8080/files/cooking-knowledge-graph.rdf
- http://streamhub:9292/streamhub/streams/temperaturestream
- http://streamhub:9292/streamhub/streams/pizzastream

In [175]:
jasper.delete(qid)

'Not Found Resource with id [temp]'

### Query parameters

In [58]:
### USELESS

qid = "cooking"
tbox = "https://protege.stanford.edu/ontologies/pizza/pizza.owl"
frmt = "JSON"
body = '''
SELECT ?x ?t
FROM NAMED WINDOW <wwin> ON <http://streamhub:9292/streamhub/streams/pizzastream> [RANGE PT15S STEP PT5S]
FROM NAMED WINDOW <tempewin> ON <http://streamhub:9292/streamhub/streams/temperaturestream> [RANGE PT15S STEP PT5S]
FROM NAMED <http://notebook:8888/files/cooking-knowledge-graph.rdf#>
    WHERE {
        {
            WINDOW <wwin> { ?x a <http://www.co-ode.org/ontologies/pizza/pizza.owl#Pizza> . }
            
            WINDOW <tempewin> {?a <http://www.w3.org/ns/sosa/hasResult> ?t .}
        }
    }
'''

In [201]:
### USELESS

qid = "cooking"
tbox = "https://protege.stanford.edu/ontologies/pizza/pizza.owl"
frmt = "JSON"
body = '''
CONSTRUCT { ?x ?x ?x}
FROM NAMED WINDOW <wwin> ON <http://streamhub:9292/streamhub/streams/pizzastream> [RANGE PT4S STEP PT1S]
    WHERE {
        WINDOW ?win { ?x <http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#MozzarellaTopping>.
                        ?x <http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#TomatoTopping>.
                        FILTER NOT EXISTS {
                        ?x <http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#PeperoniSausageTopping>.
                        }
                }
}
'''

In [165]:
### USELESS

qid = "cooking"
tbox = "https://protege.stanford.edu/ontologies/pizza/pizza.owl"
frmt = "JSON"
body = '''
SELECT ?x ?y ?z
FROM NAMED WINDOW <wwin> ON <http://streamhub:9292/streamhub/streams/pizzastream> [RANGE PT4S STEP PT1S]
FROM NAMED <http://notebook:8888/files/cooking-knowledge-graph.rdf#>
WHERE {
    { WINDOW <wwin> {?x <http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#MozzarellaTopping>. } }
    UNION
    { WINDOW <wwin> {?y <http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#TomatoTopping>.} }
    UNION
    { WINDOW <wwin> {?z <http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#PeperoniSausageTopping>.} }
}
    
    
'''

In [659]:
# USELESS

qid = "cooking"
tbox = "https://protege.stanford.edu/ontologies/pizza/pizza.owl"
frmt = "JSON"
body = '''
SELECT ?x ?y
FROM NAMED WINDOW <pw> ON <http://streamhub:9292/streamhub/streams/pizzastream> [RANGE PT15S STEP PT5S]
FROM NAMED WINDOW <tw> ON <http://streamhub:9292/streamhub/streams/temperaturestream> [RANGE PT15S STEP PT5S]
WHERE {
    WINDOW <pw> {
        ?x <http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> ?y.
            }
#    WINDOW <tw> {
#        ?q a <http://www.w3.org/ns/sosa/Observation>.
#        ?q <http://www.w3.org/ns/sosa/hasFeatureOfInterest> ?p.
#        ?p <http://www.w3.org/1999/02/22-rdf-syntax-ns#ID> ?s.
#    }  
}
'''

In [253]:
#Here we can distinguish the two types of pizza
qid = "cooking"
tbox = "https://protege.stanford.edu/ontologies/pizza/pizza.owl"
frmt = "JSON"
body = '''
SELECT ?x ?z
FROM NAMED WINDOW <pw1> ON <http://streamhub:9292/streamhub/streams/pizzastream> [RANGE PT15S STEP PT5S]
FROM NAMED WINDOW <pw2> ON <http://streamhub:9292/streamhub/streams/pizzastream> [RANGE PT15S STEP PT5S]
FROM NAMED WINDOW <pw3> ON <http://streamhub:9292/streamhub/streams/pizzastream> [RANGE PT15S STEP PT5S]
FROM NAMED WINDOW <tw> ON <http://streamhub:9292/streamhub/streams/temperaturestream> [RANGE PT15S STEP PT5S]
WHERE {
    {WINDOW <pw1> {
        ?x a <http://www.co-ode.org/ontologies/pizza/pizza.owl#Pizza>;
             <http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping>  <http://www.co-ode.org/ontologies/pizza/pizza.owl#TomatoTopping> ;
             <http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping>  <http://www.co-ode.org/ontologies/pizza/pizza.owl#MozzarellaTopping> .
         FILTER NOT EXISTS {
                        ?x <http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#PeperoniSausageTopping>.
                        }
        }}
    UNION
    {WINDOW <pw3> {
        ?z a <http://www.co-ode.org/ontologies/pizza/pizza.owl#Pizza>;
             <http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping>  <http://www.co-ode.org/ontologies/pizza/pizza.owl#PeperoniSausageTopping> .
    } }
    
} 
'''

In [206]:
#Here we have the temperature
qid = "temp"
tbox = "https://protege.stanford.edu/ontologies/pizza/pizza.owl"
frmt = "JSON"
body = '''
SELECT ?id ?pizza ?value
FROM NAMED WINDOW <tw> ON <http://streamhub:9292/streamhub/streams/temperaturestream> [RANGE PT15S STEP PT5S]
WHERE {
    WINDOW <tw> {
        ?id a <http://www.w3.org/ns/sosa/Observation>;
        <http://www.w3.org/ns/sosa/hasFeatureOfInterest> ?pizza;
        <http://www.w3.org/ns/sosa/hasResult> ?res.
        ?res <http://qudt.org/1.1/schema/qudt#numericValue> ?value.
      
    }
} 
'''

In [214]:
#Here we can distinguish the two types of pizza
qid = "cooking"
tbox = "https://protege.stanford.edu/ontologies/pizza/pizza.owl"
frmt = "JSON"
body = '''
SELECT ?x ?z ?value
FROM NAMED WINDOW <pw1> ON <http://streamhub:9292/streamhub/streams/pizzastream> [RANGE PT15S STEP PT5S]
FROM NAMED WINDOW <pw2> ON <http://streamhub:9292/streamhub/streams/pizzastream> [RANGE PT15S STEP PT5S]
FROM NAMED WINDOW <pw3> ON <http://streamhub:9292/streamhub/streams/pizzastream> [RANGE PT15S STEP PT5S]
FROM NAMED WINDOW <tw> ON <http://streamhub:9292/streamhub/streams/temperaturestream> [RANGE PT15S STEP PT5S]
WHERE {
    {WINDOW <pw1> {
        ?x a <http://www.co-ode.org/ontologies/pizza/pizza.owl#Pizza>;
             <http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping>  <http://www.co-ode.org/ontologies/pizza/pizza.owl#TomatoTopping> ;
             <http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping>  <http://www.co-ode.org/ontologies/pizza/pizza.owl#MozzarellaTopping> .
         FILTER NOT EXISTS {
                        ?x <http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#PeperoniSausageTopping>.
                        }
        }}
    UNION
    {WINDOW <pw3> {
        ?z a <http://www.co-ode.org/ontologies/pizza/pizza.owl#Pizza>;
             <http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping>  <http://www.co-ode.org/ontologies/pizza/pizza.owl#PeperoniSausageTopping> .
    } }
    
    {WINDOW <tw> {
        ?id a <http://www.w3.org/ns/sosa/Observation>;
        <http://www.w3.org/ns/sosa/hasFeatureOfInterest> ?pizza;
        <http://www.w3.org/ns/sosa/hasResult> ?res.
        ?res <http://qudt.org/1.1/schema/qudt#numericValue> ?value.
      
    } }   
    
} 
'''

In [215]:
jasper.create(qid, body, tbox, frmt)

http://jasper:8181/jasper/queries/cooking

In [89]:
cooking_q = jasper.getq(qid)
cooking_q

http://jasper:8181/jasper/queries/temp

In [90]:
jasper.streams()

[http://streamhub:9292/streamhub/streams/temperaturestream,
 http://jasper:8181/jasper/streams/temp]

In [216]:
r = jasper.expose(qid, 'HTTP')#, retention=5)
#r = jasper.expose(qid,'WEBSOCKET')

In [217]:
endpoint = r.endpoints()[0]
endpoint

GET http://jasper:8183/jasper/streams/cooking/observers/1215922378

In [219]:
resp = endpoint.call()
resp

[
    {
        "head": {
            "vars": [
                "x",
                "z",
                "value"
            ]
        },
        "results": {
            "bindings": [
                {
                    "x": {
                        "type": "uri",
                        "value": "http://jasper:8181/6fcb2a20-9e54-49d7-abaf-6fccd3ca9b86"
                    },
                    "value": {
                        "type": "literal",
                        "datatype": "http://www.w3.org/2001/XMLSchema#decimal",
                        "value": "275.28544242454376"
                    }
                },
                {
                    "x": {
                        "type": "uri",
                        "value": "http://jasper:8181/6fcb2a20-9e54-49d7-abaf-6fccd3ca9b86"
                    },
                    "value": {
                        "type": "literal",
                        "datatype": "http://www.w3.org/2001/XMLSchema#decimal",
               

In [204]:
jasper.streams()

[]

In [125]:
jasper.delete(qid)

{
    "@id": "queries/temp",
    "@type": "vprov:Task",
    "prov:generated": {
        "@id": "streams/temp"
    },
    "prov:uses": {
        "@id": "http://streamhub:9292/streamhub/streams/temperaturestream"
    },
    "@context": {
        "@base": "http://jasper:8181/jasper/",
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "vsd": "http://w3id.org/rsp/vocals-sd#",
        "vocals": "http://w3id.org/rsp/vocals#",
        "xsd": "http://www.w3.org/2001/XMLSchema#",
        "format": "http://www.w3.org/ns/formats/",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "vprov": "http://w3id.org/rsp/vocals-prov#",
        "dcat": "http://www.w3.org/ns/dcat#",
        "prov": "http://www.w3.org/ns/prov#"
    }
}